# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break
    
from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class, pn_aligned_by_seg, pn_aligned_by_event
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils, ml_methods_utils
from planning_analysis.show_planning import nxt_ff_utils, show_planning_utils
from neural_data_analysis.neural_analysis_tools.gpfa_methods import elephant_utils, fit_gpfa_utils, plot_gpfa_utils, gpfa_helper_class
from neural_data_analysis.neural_analysis_tools.align_trials import time_resolved_regression, time_resolved_gpfa_regression,plot_time_resolved_regression
from neural_data_analysis.neural_analysis_tools.align_trials import align_trial_utils
from decision_making_analysis.compare_GUAT_and_TAFT import find_GUAT_or_TAFT_trials

from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_psth import core_stops_psth, get_stops_utils, psth_postprocessing, psth_stats, compare_events, dpca_utils
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_fit import stop_glm_fit, cv_stop_glm, glm_fit_utils, variance_explained
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_plotting import plot_spikes, plot_glm_fit, plot_tuning_func
from neural_data_analysis.design_kits.design_around_event import event_binning, stop_design, cluster_design, design_checks
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_hyperparams import compare_glm_configs, glm_hyperparams_class
from neural_data_analysis.topic_based_neural_analysis.ff_visibility import ff_vis_epochs, vis_design
from neural_data_analysis.neural_analysis_tools.glm_tools.glm_decoding_tools import glm_decoding_llr, glmglm_decoding_llrglm_decoding_llrglm_decoding_llr


import sys
import math
import gc
import subprocess
from pathlib import Path

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi
import cProfile
import pstats

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr
import statsmodels.api as sm

# Neuroscience specific imports
import neo
import rcca

# To fit gpfa
import numpy as np
from importlib import reload
from scipy.integrate import odeint
import quantities as pq
import neo
from elephant.spike_train_generation import inhomogeneous_poisson_process
from elephant.gpfa import GPFA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from elephant.gpfa import gpfa_core, gpfa_util

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

print("done")


%load_ext autoreload
%autoreload 2
%matplotlib inline

# retrieve data

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0413"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0321"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0329"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0403"

In [ ]:
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0312"

# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0316"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0327"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0328"

In [ ]:
reduce_y_var_lags = False
planning_data_by_point_exists_ok = True
y_data_exists_ok = True

pn = pn_aligned_by_event.PlanningAndNeuralEventAligned(raw_data_folder_path=raw_data_folder_path)
pn.prep_data_to_analyze_planning(planning_data_by_point_exists_ok=planning_data_by_point_exists_ok)

if not hasattr(pn, 'spikes_df'):
    pn.retrieve_or_make_monkey_data()
    pn.spikes_df = neural_data_processing.make_spikes_df(pn.raw_data_folder_path, pn.ff_caught_T_sorted,
                                                            sampling_rate=pn.sampling_rate)

# FF visible

In [ ]:
pn.make_or_retrieve_ff_dataframe()
df = pn.ff_dataframe.copy()

# minimal: detect runs, no merging (each run is its own cluster)
df2 = ff_vis_epochs.compute_visibility_runs_and_clusters(
    df, ff_col='ff_index', t_col='point_index', time_col='time', vis_col='visible',
    chunk_merge_gap=0.05,    # seconds: merge *raw* runs into chunks if gap <= this
    cluster_merge_gap=1
)

df2 = ff_vis_epochs.add_global_visibility_bursts(df2, global_merge_gap=0.25)
#df2 = ff_vis_epochs.add_global_vis_cluster_id(df2, group_cols=None, nullable_int=True)
df2 = ff_vis_epochs.add_global_vis_chunk_id(df2, group_cols=None, nullable_int=True)
#df2 = ff_vis_epochs.add_global_vis_cluster_id(df2, group_cols=None, nullable_int=True)

vis_df = df2.loc[df2['visible'] == 1].copy()


In [ ]:
# based on any ff visible
sequential_vis_df = vis_df[['ff_index', 'ff_vis_start_time', 'ff_vis_end_time',
                           'global_vis_chunk_id', 'global_burst_id', 'global_burst_start_time','global_burst_end_time',
                           'global_burst_duration','global_burst_size',
                           #'global_burst_prev_start_time','global_burst_prev_end_time'
                           ]].drop_duplicates().reset_index(drop=True)
sequential_vis_df = sequential_vis_df.sort_values('ff_vis_start_time').reset_index(drop=True)
sequential_vis_df['prev_time'] = sequential_vis_df['ff_vis_start_time'].shift(1)
sequential_vis_df['next_time'] = sequential_vis_df['ff_vis_start_time'].shift(-1)


new_seg_info = event_binning.pick_event_window(sequential_vis_df,
                                                event_time_col='ff_vis_start_time',
                                                prev_event_col='prev_time',
                                                next_event_col='next_time',
                                                pre_s=1, post_s=1.0, min_pre_bins=3, min_post_bins=3, bin_dt=0.04)
new_seg_info['event_id'] = new_seg_info['global_vis_chunk_id']
new_seg_info['event_time'] = new_seg_info['ff_vis_start_time']
new_seg_info

In [ ]:
events_with_stats = sequential_vis_df[['global_vis_chunk_id', 'global_burst_id', 'ff_vis_start_time', 'ff_vis_end_time']].copy()
events_with_stats = sequential_vis_df.rename(columns={'global_vis_chunk_id': 'event_id', 
                                           'global_burst_id': 'event_cluster_id', 
                                           'ff_vis_start_time': 'event_id_start_time', 
                                           'ff_vis_end_time': 'event_id_end_time'})

In [ ]:
events_with_stats

# Prepare for GLM

## binned_feats

In [ ]:
reload(event_binning)
reload(stop_design)
reload(cluster_design)
reload(design_checks)
reload(stop_glm_fit)
reload(glm_fit_utils)
reload(cv_stop_glm)
reload(plot_tuning_func)

In [ ]:
# 1) Build bins from your event windows (gaps allowed)
bin_dt = 0.04
bins_2d, meta = event_binning.event_windows_to_bins2d(new_seg_info, bin_dt=bin_dt, only_ok=False)

# 2) Get overlap assignments once
sample_idx, bin_idx_array, dt_array, n_bins = event_binning.build_bin_assignments(
    pn.monkey_information['time'].to_numpy(),
    bins_2d,
)

# 3) Subselect raw samples once
monkey_information_sub = pn.monkey_information.iloc[sample_idx].copy()

# 3a) One pass to get exposure and used_bins
_dummy, exposure, used_bins = event_binning.bin_timeseries_weighted(
    monkey_information_sub['time'].to_numpy(),  # any column of same length works
    dt_array, bin_idx_array, how='mean'
)

# 3b) Aggregate features with the SAME assignments
def agg_feat(col):
    vals = monkey_information_sub[col].to_numpy()
    out, _exp, _ub = event_binning.bin_timeseries_weighted(vals, dt_array, bin_idx_array, how='mean')
    # Defensive checks: exposure/used_bins should match
    assert np.shares_memory(_exp, exposure) or np.allclose(_exp, exposure)
    assert np.array_equal(_ub, used_bins)
    return out

binned_feats = pd.DataFrame({
    'accel':     agg_feat('accel'),
    'speed':     agg_feat('speed'),
    'ang_speed': agg_feat('ang_speed'),
})

# Clean NaNs (optional: choose your policy)
binned_feats = binned_feats.replace([np.inf, -np.inf], np.nan).fillna(0.0)

# 3c) Keep bins with exposure > 0
mask_used = exposure > 0
pos = used_bins[mask_used]
binned_feats = binned_feats.iloc[mask_used].reset_index(drop=True)

meta_by_bin = meta.set_index('bin').sort_index()
meta_used   = meta_by_bin.loc[pos].reset_index()   # rows now match binned_feats 


# 4) Bin spikes per cluster across ALL bins, then slice by pos
spike_counts, cluster_ids = event_binning.bin_spikes_by_cluster(
    pn.spikes_df, bins_2d, time_col='time', cluster_col='cluster'
)

# Sanity checks
assert pos.size == binned_feats.shape[0]
assert spike_counts.shape[0] >= (pos.max() + 1)

binned_spikes = pd.DataFrame(
    spike_counts[pos, :],        # slice rows to align with pos
    columns=cluster_ids,         # cluster IDs as column labels
).reset_index(drop=True)

# Build the stop-aware design block (same helper we wrote earlier)
X_event_df = stop_design.build_event_design_from_meta(
    meta=meta,
    pos=pos,
    new_seg_info=new_seg_info,
    speed_used=binned_feats['speed'].values,
    include_columns=(
        'basis', 'prepost', 'prepost*speed',
        'captured', 'basis*captured', #'prepost*captured',
        #'time_since_prev_event_pre', 'time_to_next_event_post',
        'time_since_prev_event', 'time_to_next_event',
    )
)

cluster_df = cluster_design.build_cluster_features_workflow(
    meta_used[['event_id', 'rel_center']], events_with_stats,
    rel_time_col='rel_center',
    winsor_p=0.5,
    use_midbin_progress=True,
    zscore_progress=False,   # set True if you want progress in SD units
    zscore_rel_time=True
)
cluster_feats = [
        'is_clustered',
        'event_is_first_in_cluster', 
        #'event_is_last_in_cluster',
        'prev_gap_s_z',
        'next_gap_s_z',
        'cluster_duration_s_z',
        'cluster_progress_c', 'cluster_progress_c2',
        #'log_event_cluster_size_z',      # optional
        'cluster_rel_time_s_z',          # optional (bin-level)
    ]


cols_to_add_from_event_design = [c for c in X_event_df.columns if c not in binned_feats.columns]
binned_feats.loc[:, cols_to_add_from_event_design] = X_event_df[cols_to_add_from_event_design].to_numpy()  # equivalent to .values

cols_to_add_from_cluster_design = [c for c in cluster_feats if c not in binned_feats.columns]
binned_feats.loc[:, cols_to_add_from_cluster_design] = cluster_df[cols_to_add_from_cluster_design].to_numpy()  # equivalent to .values

offset_log = np.log(np.clip(exposure[mask_used], 1e-12, None)) 





# --- usage ---
binned_feats_sc, scaled_cols = event_binning.selective_zscore(binned_feats)
binned_feats_sc = sm.add_constant(binned_feats_sc, has_constant='add')
print('Scaled columns:', scaled_cols)

df_X = binned_feats_sc.copy()
df_Y = binned_spikes.copy()


In [ ]:
pn.ff_dataframe['memory'].describe()

In [ ]:


# add more columns
binned_feats['time_rel_to_event_start'] = meta_used['rel_center']


# add ff visible/in-memory info
max_in_memory_time_since_seen = 2
ff_df_sub = pn.ff_dataframe[pn.ff_dataframe['time_since_last_vis']
                                        < max_in_memory_time_since_seen].copy()
ff_df_sub['in_memory'] = 1

for state in ['visible', 'in_memory']:
    k_ff_visible = vis_design.count_visible_from_time_df_fast(
        ff_df_sub,
        bins_2d,
        vis_col=state
    )

    any_visiblef_visible = (k_ff_visible > 0).astype('int8')

    binned_feats[f'any_visiblef_{state}'] = any_visiblef_visible[used_bins]
    binned_feats[f'k_ff_{state}'] = k_ff_visible[used_bins]


# --- usage ---
binned_feats_sc, scaled_cols = event_binning.selective_zscore(binned_feats)
binned_feats_sc = sm.add_constant(binned_feats_sc, has_constant='add')
print('Scaled columns:', scaled_cols)

df_X = binned_feats_sc.copy()
df_Y = binned_spikes.copy()

# Decoding

## original

In [ ]:
# 1) Build params from the report and align to your spike matrix:
params_df = glm_decoding_llr.params_df_from_coefs_df(report['coefs_df'])     # long → wide
params_df = glm_decoding_llr.align_params_to_Y(params_df, df_Y)              # row order = df_Y columns

# 2) Decode on all rows (no CV)
#    vis_col must match the column name you used for visibility in df_X/params_df.
llr, p_vis = glm_decoding_llr.decode_from_fitted_glm(
    df_X, 
    df_Y,
    offset_log,
    params_df=params_df,
    vis_col='any_visiblef_visible'                   # <-- change if your term is named 'visible', etc.
)


In [ ]:
df_X['any_visiblef_visible'].mean()

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, precision_recall_curve

eps = 1e-9

# If you have ground-truth per-bin labels:
auc = roc_auc_score(y_visible, llr)               # using raw LLR is fine
aupr = average_precision_score(y_visible, p_vis)  # PR-AUC often more informative with class imbalance
print(f"AUC={auc:.3f}, PR-AUC={aupr:.3f}")

# Pick an operating point (threshold)
fpr, tpr, thr = roc_curve(y_visible, llr)
# Example choices:
# - Youden J (maximizes tpr - fpr)
j_idx = np.argmax(tpr - fpr)
thr_llr = thr[j_idx]

# Hard labels
y_hat = (llr >= thr_llr).astype(int)


In [ ]:
# with prior

prior = y_visible.mean()                     # or any prior you believe
logit_prior = np.log((prior+eps)/(1-prior+eps))
p_post = 1 / (1 + np.exp(-(llr + logit_prior)))  # posterior with prior

import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, precision_recall_curve

eps = 1e-9

# If you have ground-truth per-bin labels:
auc = roc_auc_score(y_visible, llr)               # using raw LLR is fine
aupr = average_precision_score(y_visible, p_post)  # PR-AUC often more informative with class imbalance
print(f"AUC={auc:.3f}, PR-AUC={aupr:.3f}")

# Pick an operating point (threshold)
fpr, tpr, thr = roc_curve(y_visible, llr)
# Example choices:
# - Youden J (maximizes tpr - fpr)
j_idx = np.argmax(tpr - fpr)
thr_llr = thr[j_idx]

# Hard labels
y_hat = (llr >= thr_llr).astype(int)


In [ ]:
# Build dt from your bins (if irregular)
dt = np.ones(len(meta_used), dtype=float) * 0.04
groups = meta_used['event_id']


for col in ['any_visiblef_visible', 'any_visiblef_in_memory']:
    y_visible = binned_feats[col].values
    p = y_visible.mean()
    print(f"{col} prevalence = {p:.2f}")
    
    # # If your spikes/feats are DataFrames, you can pass them directly,
    # # or use .to_numpy() if you prefer explicit arrays.
    # auc_mean, auc_std = glm_decoding_llr.cv_decode_glm(
    #     binned_spikes,      # (T, N) DataFrame or array
    #     binned_feats,       # (T, F) DataFrame or array
    #     y_visible,          # (T,)
    #     groups,             # (T,)
    #     dt,                 # float or (T,)
    #     alpha=0.1
    # )
    # print(auc_mean, auc_std)

    auc_mean, auc_std = glm_decoding_llr.cv_decode_glm(
        binned_spikes,      # DataFrame or ndarray (T,N)
        binned_feats,       # DataFrame or ndarray (T,F)
        y_visible,          # ndarray (T,)
        groups,             # ndarray (T,)
        dt,                 # float or ndarray (T,)
        alpha=0.1,          # ridge strength
        scale_X=True,
        verbose=True
    )
    print(f"GLM-LLR AUC: {auc_mean:.3f} ± {auc_std:.3f}")



    # auc_mean, auc_std = glm_decoding_llr.cv_decode_glm(binned_spikes, binned_feats, y_visible, groups, dt, alpha=0.1)
    # print(f"GLM-LLR AUC: {auc_mean:.3f} ± {auc_std:.3f}")



In [ ]:
binned_feats['any_visiblef_visible'].mean()

In [ ]:
binned_feats['any_visiblef_in_memory'].mean()

In [ ]:
for col in ['any_visiblef_visible', 'any_visiblef_in_memory']:
    y_visible = binned_feats[col].values
    p = y_visible.mean()
    print(f"{col} prevalence = {p:.2f}")
    best_model, cv_auc, all_auc, best_params = glm_decoding_llr.decode_visible_with_lr(
        binned_spikes, y_visible, groups, dt=bin_dt, try_pca=False
    )
    print(best_params, f'CV AUC={cv_auc:.3f}, all-data AUC={all_auc:.3f}')


In [ ]:
y_visible = binned_feats[col].values

## pipeline

In [ ]:
out = glm_decoding_llr.run_full_decode_pipeline(
    report, df_X, df_Y, offset_log, bins_2d,
    vis_col='any_visiblef_visible',
    prior=None,          # or a base rate like 0.15
    use_hmm=True, tau_vis=0.5, tau_inv=0.7, pi0=0.05,
    y_visible=y_visible, # optional
    fpr_target=None      # or e.g. 0.05 for ~5% FPR
)
print(f"AUC={out['auc']:.3f}, PR-AUC={out['ap']:.3f}, tau={out['tau']:.3f}")
print("First 5 episodes (HMM):", out['episodes_hmm'][:5])


In [ ]:
auc_obs, pval, null = auc_permutation_test(y_visible, p_post_f, groups=g_f, n_perm=1000, rng=0, mask=mask_guard_f)
print(f"Observed AUC={auc_obs:.3f}, permutation p≈{pval:.4f}")

In [ ]:
binned_feats.shape

## permutation

In [ ]:
auc_obs, pval, null = glm_decoding_llr.auc_permutation_test(
    y_visible, p_post, groups=groups, n_perm=5000, rng=0,
    progress=True, desc="Permutations"
)

mean_auc, lo, hi, aucs = glm_decoding_llr.auc_block_bootstrap_ci(
    y_visible, p_post, groups=groups, n_boot=5000, rng=1,
    progress=True, desc="Bootstraps"
)

print("=== Decoding significance ===")
print(f"Observed AUC        : {auc_obs:.3f}")
print(f"Permutation p-value : {pval:.4g}  (n={len(null)})")
print(f"Bootstrap mean AUC  : {mean_auc:.3f}")
print(f"Bootstrap 95% CI    : [{lo:.3f}, {hi:.3f}]  (n={len(aucs)})")



## cv

In [ ]:
res = glm_decoding_llr.cv_decode_with_glm_report(
    df_X=df_X,                 # design (must include 'any_ff_visible')
    df_Y=df_Y,                 # spikes (T x N), columns are unit IDs
    y=y_visible,               # (T,) 0/1
    groups=groups,             # (T,) session/episode IDs for GroupKFold
    offset_log=offset_log,     # scalar or (T,)
    fit_fn=stop_glm_fit.glm_mini_report,
    fit_kwargs=dict(cov_type='HC1', fast_mle=True, do_inference=False, make_plots=False, show_plots=False),
    bins_2d=bins_2d[used_bins],           # to enable guard-band evaluation
    vis_col='any_ff_visible',
    n_splits=5,
    standardize=False,         # set True if you want z-scoring of features on train only
    guard=0.05                 # None to disable; else ignores test bins near edges
)

print(f"GroupKFold AUC: {res['auc_mean']:.3f} ± {res['auc_std']:.3f} | "
      f"PR-AUC: {res['pr_mean']:.3f} ± {res['pr_std']:.3f} | folds={res['n_splits']}")
for m in res['fold_metrics']:
    print(f"fold {m['fold']}: AUC={m['auc']:.3f}, PR-AUC={m['pr_auc']:.3f}, n_test={m['n_test']}, kept={m['n_kept']}")


In [ ]:
y_visible.mean()

## hmm?

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score

def runs_from_binary(y):
    y = np.asarray(y, int)
    edges = np.flatnonzero(np.diff(np.r_[0, y, 0]) != 0)
    starts, ends = edges[::2], edges[1::2]
    return starts, ends

def estimate_taus_from_labels(y, bins_2d):
    # medians of visible runs and invisible gaps (in seconds)
    dt = bins_2d[:,1] - bins_2d[:,0]
    s1, e1 = runs_from_binary(y)        # visible runs
    vis_durs = (bins_2d[e1-1,1] - bins_2d[s1,0]) if len(s1) else np.array([])
    # gaps are runs of 0 between visible runs
    y0 = 1 - y
    s0, e0 = runs_from_binary(y0)
    gap_durs = (bins_2d[e0-1,1] - bins_2d[s0,0]) if len(s0) else np.array([])
    tau_vis = float(np.median(vis_durs)) if vis_durs.size else 0.5
    tau_inv = float(np.median(gap_durs)) if gap_durs.size else 0.7
    return max(tau_vis, 1e-3), max(tau_inv, 1e-3)

def hmm_binary_smoother(llr, dt, tau_inv=0.7, tau_vis=0.5, pi0=0.05):
    from scipy.special import logsumexp
    llr = np.asarray(llr, float); dt = np.asarray(dt, float)
    if dt.ndim == 0: dt = np.full_like(llr, float(dt))
    T = llr.size
    L = np.vstack([-0.5*llr, +0.5*llr]).T
    p11 = np.clip(np.exp(-dt/max(tau_vis,1e-6)), 1e-6, 1-1e-6)
    p00 = np.clip(np.exp(-dt/max(tau_inv,1e-6)), 1e-6, 1-1e-6)
    p10 = 1.0 - p11; p01 = 1.0 - p00
    logA00,logA01,logA10,logA11 = np.log(p00),np.log(p01),np.log(p10),np.log(p11)
    log_pi = np.log([1-0.05, 0.05])
    # forward
    la = np.empty((T,2)); la[0]=log_pi+L[0]
    for t in range(1,T):
        a0 = logsumexp([la[t-1,0]+logA00[t-1], la[t-1,1]+logA10[t-1]]); 
        a1 = logsumexp([la[t-1,0]+logA01[t-1], la[t-1,1]+logA11[t-1]]);
        la[t,0]=a0+L[t,0]; la[t,1]=a1+L[t,1]
    # backward
    lb = np.zeros((T,2))
    for t in range(T-2,-1,-1):
        b0 = logsumexp([logA00[t]+L[t+1,0]+lb[t+1,0], logA01[t]+L[t+1,1]+lb[t+1,1]])
        b1 = logsumexp([logA10[t]+L[t+1,0]+lb[t+1,0], logA11[t]+L[t+1,1]+lb[t+1,1]])
        lb[t,0]=b0; lb[t,1]=b1
    lg = la+lb; lg -= logsumexp(lg, axis=1, keepdims=True)
    return np.exp(lg)[:,1]  # posterior P(visible)

def grid_search_hmm(llr_or_p, bins_2d, y_true=None, use_probs=True, base_taus=None, mult=(0.5, 0.75, 1.0, 1.5, 2.0)):
    dt = (bins_2d[:,1]-bins_2d[:,0])
    if use_probs:
        eps=1e-12
        llr = np.log(llr_or_p+eps) - np.log(1-llr_or_p+eps)
    else:
        llr = np.asarray(llr_or_p, float)
    if base_taus is None:
        tau_v, tau_i = estimate_taus_from_labels((y_true if y_true is not None else (llr>0).astype(int)), bins_2d)
    else:
        tau_v, tau_i = base_taus
    best = {"auc": -np.inf, "tau_vis": None, "tau_inv": None, "p": None}
    for mv in mult:
        for mi in mult:
            p = hmm_binary_smoother(llr, dt, tau_inv=tau_i*mi, tau_vis=tau_v*mv)
            if y_true is None:
                # no labels: prefer smoother (lower total variation)
                tv = np.sum(np.abs(np.diff(p)))
                score = -tv
            else:
                score = roc_auc_score(y_true, p)
            if score > best["auc"]:
                best = {"auc": float(score), "tau_vis": tau_v*mv, "tau_inv": tau_i*mi, "p": p}
    return best

# run it
base_tau = estimate_taus_from_labels(y_visible, bins_2d)
best = grid_search_hmm(p_post, bins_2d, y_true=y_visible, use_probs=True, base_taus=base_tau)
print(f"[HMM tuned] AUC={best['auc']:.3f}, tau_vis={best['tau_vis']:.3f}s, tau_inv={best['tau_inv']:.3f}s")
p_smooth = best['p']


# GLM

## check df

In [ ]:
X_pruned = design_checks.check_design(binned_feats_sc)

## regular

In [ ]:
reload(glm_fit_utils)
reload(stop_glm_fit)

report = stop_glm_fit.glm_mini_report(
    df_X=df_X, df_Y=df_Y, offset_log=offset_log,
    cov_type='HC1',            # or 'nonrobust' for even faster
    fast_mle=True,             # << use the ultra-fast path
    do_inference=True,        # skip FDR/ratios/pop-tests
    make_plots=True,          # skip figure creation
    show_plots=True,          # nothing to display
)
